In [1]:
#
# --- DataProcesser ---
#
# Add functions dp.add_STUFF_to_TEAM_BOX (add ALL DATA prior to modeling class/phase)
#   For dp.create_processed_team_box_and_add_season_schedule
#      Add n_days_rest (1-k), n_days_travel (0-k) (don't count current)
#         Should these be one-hots?!!!  Check online
#
#
# --- DataModeler ---
#
# Add best ML algos to dm.train_model
#   Add those 3 weka algos
#   Hunt other candidates based on problem type, amount of data
#


from src.DataProcessor import DataProcessor
from src.DataModeler import DataModeler

In [2]:
dp = DataProcessor()

dp.update_all_data()

#dp.scrape_data_player_box_scores()
#dp.scrape_data_team_box_scores()
#dp.scrape_data_season_schedule()

#dp.create_processed_team_box_and_add_season_schedule()
#dp.add_fgp_tpp_tr_to_processed_team_box()

#dp.write_complete_processed_team_box()


Scraping player box score data.

Player box score data found for season: 1999-2000
Player box score data found for season: 2000-2001
Player box score data found for season: 2001-2002
Player box score data found for season: 2002-2003
Player box score data found for season: 2003-2004
Player box score data found for season: 2004-2005
Player box score data found for season: 2005-2006
Player box score data found for season: 2006-2007
Player box score data found for season: 2007-2008
Player box score data found for season: 2008-2009
Player box score data found for season: 2009-2010
Player box score data found for season: 2010-2011
Player box score data found for season: 2011-2012
Player box score data found for season: 2012-2013
Player box score data found for season: 2013-2014
Player box score data found for season: 2014-2015
Player box score data found for season: 2015-2016
Player box score data found for season: 2016-2017
Player box score data found for season: 2017-2018
Player box score

In [2]:
dm = DataModeler()

In [4]:
dm.load_data()
dm.set_feats_and_label()
dm.set_training_params(n_models=3)


Loading model from processed data.



In [6]:
#model_type = "decision_tree_classifier"
model_type = "decision_tree_regressor"

dm.train_and_test_model(model_type=model_type)

Training and testing model : 1 of 3
Training and testing model : 2 of 3
Training and testing model : 3 of 3
Mean : 0.713765
Var  : 0.000020
